In [1]:
# mounting drive
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
%cd drive/MyDrive/BullingerDigitalLMFootnotes/git_repo


/content/drive/MyDrive/BullingerDigitalLMFootnotes/git_repo


In [3]:
!pip install -q langchain
!pip install -q torch
!pip install -q transformers
!pip install -q sentence-transformers
!pip install -q datasets
!pip install -q faiss-cpu
!pip install -U langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.4/404.4 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8

In [4]:
from langchain.document_loaders import HuggingFaceDatasetLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from transformers import AutoTokenizer, pipeline
from langchain import HuggingFacePipeline
from langchain.chains import RetrievalQA

In [35]:
# Define the path to the pre-trained model you want to use
modelPath = "dbmdz/bert-base-german-uncased"

# Create a dictionary with model configuration options, specifying to use the CPU for computations
model_kwargs = {'device':'cpu'}

# Create a dictionary with encoding options, specifically setting 'normalize_embeddings' to False
encode_kwargs = {'normalize_embeddings': False}

# Initialize an instance of HuggingFaceEmbeddings with the specified parameters
embeddings = HuggingFaceEmbeddings(
    model_name=modelPath,     # Provide the pre-trained model's path
    model_kwargs=model_kwargs, # Pass the model configuration options
    encode_kwargs=encode_kwargs # Pass the encoding options
)

config.json:   0%|          | 0.00/433 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/442M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/247k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [36]:
import numpy as np
def cosine_similarity(vector1, vector2):
    """
    Calculate the cosine similarity between two vectors.

    Args:
        vector1: The first vector (list of numbers).
        vector2: The second vector (list of numbers).

    Returns:
        The cosine similarity between the two vectors (a float between -1 and 1).
    """

    # Convert the lists to NumPy arrays for efficient calculations.
    vector1 = np.array(vector1)
    vector2 = np.array(vector2)

    # Calculate the dot product of the two vectors.
    dot_product = np.dot(vector1, vector2)

    # Calculate the magnitudes (Euclidean norms) of the two vectors.
    magnitude1 = np.linalg.norm(vector1)
    magnitude2 = np.linalg.norm(vector2)

    # Calculate the cosine similarity using the formula:
    # cosine_similarity = dot_product / (magnitude1 * magnitude2)
    similarity = dot_product / (magnitude1 * magnitude2)

    return similarity

In [37]:
text1 = "Die Katze sizt auf dem Tisch"
text2 = "Die Katze ist auf dem Tisch"
text3 = "aber wenn es auch wirklich nichts damit zu tun hat, aber einfach länger ist? Da kriegt es automatisch einen besseren score... Oder naja also besser als 0.43 wird es nicht? Ne, es wird sogar wieder schlechter, das ist gut"
query_result1 = embeddings.embed_query(text1)
query_result2 = embeddings.embed_query(text2)
query_result3 = embeddings.embed_query(text3)
print(cosine_similarity(query_result1, query_result2), cosine_similarity(query_result1, query_result3))

0.9065462109309015 0.5915995122275589


In [38]:
# loading the texts
from langchain_community.document_loaders import TextLoader
import json
from tqdm import tqdm

with open("../data/strat_sample.json", "r", encoding="utf-8") as f:
    strat_sample = json.load(f)
    letter_ids = strat_sample["train"]

data = []
for letter_id in tqdm(letter_ids[:100]):
  loader = TextLoader(f"../fine_tuning_data/hbbw_regesten/{letter_id}.txt")
  # Todo: make custom loader, also maybe already chunk it into sentences? Thus we can make smaller documents
  document = loader.load()[0]
  document.metadata["id"] = letter_id
  data.append(document)


100%|██████████| 100/100 [00:00<00:00, 267.68it/s]


In [ ]:
# loading zwingliana
import os
for root, dirs, files in os.walk("../data/strat_sample"):
  for file in files:
    file_path = os.path.join(root, file)

In [16]:
data[0]

Document(metadata={'source': '../fine_tuning_data/hbbw_regesten/11143.txt', 'id': '11143'}, page_content='Johannes Rhellikan (Müller aus Rellikon) / Bern an Heinrich Bullinger, Leo Jud, Kaspar Megander (Großmann), 12. Februar 1538\n\n\n \nBerichtet, was seit den Verhandlungen der  Aargauer  Pfarrer mit\ndem Rat [über die Revision des Katechismus] geschehen ist: Diakon\n Jakob [Meyer]  wurde wegen seines\nWiderstands gegen den neuen Katechismus abgesetzt, aber auf Bitte der\n Aargauer  zur Versetzung in eine Landgemeinde begnadigt; die\nStadtpfarrer haben sich bei den Aargauern entschuldigt, ihre Vorschläge gebilligt, sie bewirtet und sich\nbeim Stadtschreiber für die Korrektur des Katechismus und für die rasche Ausfertigung des Abschieds\neingesetzt. Ursache dieser scheinbaren Wandlung der Buceraner ist ihre Furcht vor offenem Aufruhr; sie\nversuchen, die Änderung des Katechismus zu vertuschen. Inzwischen wurde der Vogt von\n Bipp  wegen eines Liedes vernommen, wobei er angab, ein Pred

In [ ]:
# Create an instance of the RecursiveCharacterTextSplitter class with specific parameters.
# It splits text into chunks of 1000 characters each with a 150-character overlap.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)

# 'data' holds the text you want to split, split the text into documents using the text splitter.
docs = text_splitter.split_documents(data)

In [39]:
docs = data

In [17]:
docs[0]

NameError: name 'docs' is not defined

In [40]:

from tqdm import tqdm
db = FAISS.from_documents(tqdm(docs), embeddings)

100%|██████████| 100/100 [00:00<00:00, 260031.25it/s]


In [42]:
question = "Jakob Meyer und katechismus"
q_embeds = embeddings.embed_query(question)
searchDocs = db.similarity_search_with_score(question)
for result in searchDocs:
  print(result)
  print("#########################################")


(Document(metadata={'source': '../fine_tuning_data/hbbw_regesten/10109.txt', 'id': '10109'}, page_content="Gervasius Schuler (Scholasticus) / [Basel] an Heinrich Bullinger, 23. März 1532\n\n\n \nBedankt sich für die übersandten Bullingerwerke, gibt eine Darstellung von\n Grynäus ' Auffassung über Kirchenzucht; schließt sich in dieser Frage\njedoch den Zürchern an.  Bonifacius Amerbach  lehnt das Abendmahl ab.\n Schuler  erwartet ungeduldig Bullingers Kommentar zum Hebräerbrief.\nGrüße.\n \n"), 226.96437)
#########################################
(Document(metadata={'source': '../fine_tuning_data/hbbw_regesten/10163.txt', 'id': '10163'}, page_content='Berchtold Haller / Bern an Heinrich Bullinger, 12. September [1532]\n\n\n \n Haller  ist wegen der durch Täufer und Katholiken gefährdeten Lage der\nReformation in  Solothurn  besorgt, bezweifelt jedoch die\nBerechtigung und Zweckmäßigkeit der Todesstrafe gegen die Täufer, zumal die Katholiken straflos ausgingen.\nEr wiederholt seine Bitte

In [43]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Assuming 'docs' contains your list of Langchain Documents
text_content = [doc.page_content for doc in docs]

# Create a TfidfVectorizer object
vectorizer = TfidfVectorizer()

# Fit the vectorizer to your documents and transform them into TF-IDF vectors
tfidf_matrix = vectorizer.fit_transform(text_content)

# Now, tfidf_matrix contains your documents in TF-IDF format.

In [49]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

# Assuming 'docs' contains your list of Langchain Documents and 'tfidf_matrix'
# is your TF-IDF matrix from the previous step

def search_tfidf(query, top_k=5):
    """Searches documents based on TF-IDF cosine similarity.

    Args:
        query: The search query string.
        top_k: The number of top results to return.

    Returns:
        A list of tuples containing (document, similarity_score) for the top k results.
    """
    # Create a TfidfVectorizer (using the same settings as before)
    vectorizer = TfidfVectorizer()
    vectorizer.fit([doc.page_content for doc in docs]) # fit to your docs

    # Transform the query into TF-IDF
    query_tfidf = vectorizer.transform([query])

    # Calculate cosine similarity between query and documents
    similarities = cosine_similarity(query_tfidf, tfidf_matrix)

    # Get the indices of the top k most similar documents
    top_indices = similarities.argsort()[0][-top_k:][::-1]

    # Return the top documents and their similarity scores
    results = [(docs[i], similarities[0][i]) for i in top_indices]
    return results


# Example usage:
query = "Wogegen hat Jakob Meyer wiederschand gezeigt??"
results = search_tfidf(query, top_k=3)

for doc, score in results:
    print(f"Document: {doc}, Similarity Score: {score}")
    print("#########################################")

Document: page_content='Oswald Myconius (Geißhüsler) / Basel an Heinrich Bullinger, 8. April 1539


 
Da  Grynäus  keine ungebundenen Bücher liest, haben sie beide Bullingers
[„De origine erroris“] noch nicht studiert. Von den Verhandlungen in
 Frankfurt  will  Myconius 
schweigen und einzig erwähnen, daß der Kaiser, der zum Krieg gegen den englischen
 König [Heinrich VIII.]  rüstet, dessen Aufnahme in den [Schmalkaldischen] Bund zu
verhindern sucht. Die feindliche Haltung der Kaiserlichen hat sich auch bei einem Zwischenfall mit
Gefolgsleuten des Großhofmeisters [ Adrian von Croy ] bei
 St. Jakob an der Birs 
gezeigt. Weiß nicht, ob sich Bürgermeister [Jakob
Meyer] wie versprochen für
 [Diethelm] Keller  eingesetzt
hat. Ob  Grynäus   Rudolf [Gwalther]  bei sich aufnehmen kann, ist angesichts der veränderten
Umstände noch unklar;  Myconius  verspricht sich für die Zukunft viel
von  Gwalther .
 
' metadata={'source': '../fine_tuning_data/hbbw_regesten/11295.txt', 'id': '11295'}, Similar

In [51]:
%pip install rank_bm25

In [57]:
from rank_bm25 import BM25Okapi
import numpy as np

tokenized_corpus = [doc.page_content.split() for doc in docs]
bm25 = BM25Okapi(tokenized_corpus)

query = "Wogegen hat Jakob Meyer wiederschand gezeigt??"
tokenized_query = query.split()
scores = bm25.get_scores(tokenized_query)

max_index = np.argmax(scores)
print(" ".join(tokenized_corpus[max_index]))


Heinrich Bullinger / Zürich an Johannes Haller d.J., 30. Januar 1547 [1] Wie aus Hallers Brief [ Nr. 2769 ] vom 25. Januar hervorgeht, sind Bullingers Briefe nicht angekommen, doch müsste Haller unterdessen wenigstens den von seinem Verwandten [ Konrad Kambli ] überbrachten Brief [vom 22. Januar; nicht erhalten] empfangen haben. Bullinger hat seinen [von Haller gewünschten] Ratschlag rechtzeitig genug mitgeteilt: Es gilt, solange in Augsburg auszuhalten, wie es Hallers Gewissen erlaubt und man für [das Evangelium] eintreten kann. Aber natürlich soll Haller sich und seine drei Kollegen [ Lorenz Meyer , Thoman Ruman und Rudolf Schwyzer d.Ä.] nicht dem Tod ausliefern, zumal auch Christus und Paulus mehrmals vor ihren Feinden geflohen sind. Wenn Haller aber in Feindeshand geriete, sollte er gemäß Mt 10, [28] nicht die fürchten, die nur den Leib, nicht aber die Seele töten können. [2] Gestern traf Hallers Brief [ Nr. 2759 vom 19. Januar] ein. Den an die Zürcher Bürgermeister Hans Rudolf Lav